### Import statements

In [14]:
import sys
sys.path.insert(0, '/Users/ahell/Documents/Python Projects/AIToolkit')

import os
from dotenv import load_dotenv, find_dotenv
from ai_toolkit import AIModel, PromptBuilder, FileReader, FileWriter, AIProcess, Function, APIRequest
from IPython.display import display, Markdown, HTML, clear_output

In [3]:
# Load in the environment variables
load_dotenv(find_dotenv())

True

In [4]:
AIModel.ALL_MODEL_NAMES

['gpt-3.5-turbo',
 'gpt-4',
 'gpt-3.5-turbo-16k',
 'claude-1',
 'claude-1-100k',
 'claude-instant-1',
 'claude-instant-1-100k',
 'respell-gpt-4-wrapper']

### Code

In [5]:
request_url = "https://dev-api.insite.ledgestone.com/api/results/comments/a710652c-b271-4499-add0-b88c14d31517/open_ended"
request_method = "GET"
request_headers = {
    "Authorization": f"Bearer {os.environ['BEARER_TOKEN']}",
    "Content-Type": "application/json"
}
comments_request = APIRequest("Comments Request").set_input(url=request_url, method=request_method, headers=request_headers)
comments_request.process()

Comments Request (APIRequest)

In [6]:
comments_function = Function("Comments List Extractor").set_input(function="extract_key", input=comments_request, key_name="comments")
comments_function.process()
comments_list = comments_function.get_output()
print(comments_list)

[{'comment': 'this is a great place to ', 'nps_category': 'promoter', 'comment_number': 1, 'prompt_response_id': '00b940bb-d597-4bff-b11c-ec62a8a0f2f7'}, {'comment': 'Need to be more caring ', 'nps_category': 'passive', 'comment_number': 2, 'prompt_response_id': '018f5d23-4831-402d-b6e3-5138c5deca13'}, {'comment': 'Improved inter department connections - events like today will aid with ', 'nps_category': 'promoter', 'comment_number': 3, 'prompt_response_id': '06e1f20f-6dc6-47ef-88c8-9668d9af0a93'}, {'comment': 'Having a schedule more than a week at a time so we could plan a life outside of work. ', 'nps_category': 'passive', 'comment_number': 4, 'prompt_response_id': '1925d6e6-a7c5-41e4-9ed0-60ad29eff5fb'}, {'comment': 'No just keep doing what you are doing ', 'nps_category': 'promoter', 'comment_number': 5, 'prompt_response_id': '1c2fcb0d-a0f5-438b-975f-f6e11c6b9801'}, {'comment': 'ADD THAT RAISE AND STOP HIRING LAZY PEOPLE.', 'nps_category': 'passive', 'comment_number': 6, 'prompt_re

In [9]:
extract_comment = Function("Extract Comment").set_input(function="extract_key", name_of_input="comment_dict", key_name="comment")
extract_comment_num = Function("Extract Comment Number").set_input(function="extract_key", name_of_input="comment_dict", key_name="comment_number")
extract_prompt_response_id = Function("Extract Prompt Response ID").set_input(function="extract_key", name_of_input="comment_dict", key_name="prompt_response_id")

dimension_prompt_file = FileReader("Dimension Analyzer File").set_input(file_path="dimension_prompt.txt")
dimension_prompt = PromptBuilder("Dimension Analyzer Prompt").set_input(template=dimension_prompt_file, comment=extract_comment)
dimension_analyzer_llm = AIModel("Dimension Analyzer LLM").set_input(model_name="gpt-3.5-turbo", prompt=dimension_prompt)
dimension_analyzer_json = Function("Dimension Analyzer JSON").set_input(function="convert_to_json", input=dimension_analyzer_llm)

convert_support_and_regognition_to_value_function = """
    analysis = get_input("analysis")
    if 'Support & Recognition' in analysis:
        analysis['Value'] = analysis.pop('Support & Recognition')
    return analysis
"""

dimension_analyzer_output_function = """
    comment_number = get_input("comment_number")
    comment = get_input("comment")
    prompt_response_id = get_input("prompt_response_id")
    analysis = get_input("analysis")
    output = list()
    for dimension, anal in analysis.items():
        output.append({
            "comment_number": comment_number,
            "comment": comment,
            "prompt_response_id": prompt_response_id,
            "dimension": dimension,
            "analysis": anal["analysis"],
            "sentiment": anal["sentiment"]
            })

    return output
"""

convert_support_and_recognition_to_value = Function("Convert Support & Recognition to Value").set_input(function=convert_support_and_regognition_to_value_function, analysis=dimension_analyzer_json)

dimension_analyzer_output = Function("Dimension Analyzer Output").set_input(function=dimension_analyzer_output_function, 
                                                                            comment_number=extract_comment_num, 
                                                                            comment=extract_comment, 
                                                                            prompt_response_id=extract_prompt_response_id, 
                                                                            analysis=convert_support_and_recognition_to_value)

dimension_analyzer = AIProcess("Dimension Analyzer")
dimension_analyzer.expose_input("comment_dict", extract_comment)
dimension_analyzer.expose_input("comment_dict", extract_comment_num)
dimension_analyzer.expose_input("comment_dict", extract_prompt_response_id)
dimension_analyzer.expose_output(dimension_analyzer_output)

Dimension Analyzer (AIProcess)

In [12]:
output = list()
for i in range(len(comments_list)):
    clear_output(wait=True)
    print(f"Processing comment {i+1} of {len(comments_list)}")
    dimension_analyzer.set_input(comment_dict=comments_list[i])
    dimension_analyzer.process()
    output.extend(dimension_analyzer.get_output())

Processing comment 24 of 24
Processing Extract Comment (Function)
Processing Extract Comment Number (Function)
Processing Extract Prompt Response ID (Function)
Processing Dimension Analyzer File (FileReader)
Processing Dimension Analyzer Prompt (PromptBuilder)
Processing Dimension Analyzer LLM (AIModel)
Processing Dimension Analyzer JSON (Function)
Processing Convert Support & Recognition to Value (Function)
Processing Dimension Analyzer Output (Function)


In [15]:
dimension_results_json = FileWriter("Save Dimension Analysis JSON").set_input(file_path="dimension_results.json", data=output)
dimension_results_json.process()
dimension_results_json.get_output()

'File successfully written'